In [8]:
!pwd

/Users/beszabo/bene/szakdolgozat/code


In [11]:
import os
import numpy as np
import pandas as pd
from lead_lag_normalized import add_normalizations, event_study_value # Assuming lead_lag_normalized.py is in 'code'
import matplotlib.pyplot as plt
from lead_lag_analysis import xcorr_by_company, plot_xcorr, event_study, plot_event
from lead_lag_normalized import event_study_value # Import event_study_value


In [21]:
REPO_ROOT = "/Users/beszabo/bene/szakdolgozat"
PANEL_DIR = os.path.join(REPO_ROOT, "data", "panels")
FIG_DIR = os.path.join(REPO_ROOT, "figures")

PANEL_CSV = os.path.join(PANEL_DIR, "company_weekly_panel_enriched.csv") # Load the correct file

panel = pd.read_csv(PANEL_CSV, parse_dates=['week_start'])
panel = add_normalizations(panel, monthly=False) #

pos_z, neg_z = event_study_value(panel, pos_feature='mean_pos', neg_feature='mean_neg', value_col='num_memes_z', window=3)
os.makedirs(FIG_DIR, exist_ok=True)

In [4]:
# Cross-correlations
for feat in ['sentiment_score', 'mean_pos', 'mean_neg', 'non_neutral_share', 'num_articles']:
    df = xcorr_by_company(panel, feat, max_lag=4)
    out = os.path.join(FIG_DIR, f'xcorr_{feat}_normalized_panel2.png') # Changed filename
    plot_xcorr(df, f'Lead-Lag: {feat} vs num_memes_z', out)
    print(f'Saved {out}')

# Z-score event study
pos_z, neg_z = event_study_value(panel, pos_feature='mean_pos', neg_feature='mean_neg', value_col='num_memes_z', window=3)
plot_event(pos_z, 'num_memes_z', 'Event: Positive news vs normalized (z) meme volume', os.path.join(FIG_DIR, 'event_pos_num_memes_z.png'))
plot_event(neg_z, 'num_memes_z', 'Event: Negative news vs normalized (z) meme volume', os.path.join(FIG_DIR, 'event_neg_num_memes_z.png'))

# print('Event-study plots saved.')

/Users/beszabo/bene/szakdolgozat/.venv/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/beszabo/bene/szakdolgozat/.venv/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3066: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/Users/beszabo/bene/szakdolgozat/.venv/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/beszabo/bene/szakdolgozat/.venv/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3066: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/Users/beszabo/bene/szakdolgozat/.venv/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/beszabo/bene/szakdolgozat/.venv/lib/python3.13/site-packages/numpy/lib/_function_ba

Saved /Users/beszabo/bene/topicality-online/figures/xcorr_sentiment_score_normalized_panel2.png
Saved /Users/beszabo/bene/topicality-online/figures/xcorr_mean_pos_normalized_panel2.png
Saved /Users/beszabo/bene/topicality-online/figures/xcorr_mean_neg_normalized_panel2.png
Saved /Users/beszabo/bene/topicality-online/figures/xcorr_non_neutral_share_normalized_panel2.png


/Users/beszabo/bene/szakdolgozat/.venv/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/beszabo/bene/szakdolgozat/.venv/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3066: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/Users/beszabo/bene/szakdolgozat/.venv/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/beszabo/bene/szakdolgozat/.venv/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3066: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/Users/beszabo/bene/szakdolgozat/.venv/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/beszabo/bene/szakdolgozat/.venv/lib/python3.13/site-packages/numpy/lib/_function_ba

KeyError: 'num_articles_L1'

In [13]:
import re


def _sanitize_filename(name: str) -> str:
    return re.sub(r"[^A-Za-z0-9_-]+", "_", str(name)).strip("_")

# plot event study with 95% CI
def plot_event_ci(df: pd.DataFrame, value_col: str, title: str, out_path: str):
    if df is None or df.empty:
        return
    agg = df.groupby('tau')[value_col].agg(['mean', 'std', 'count']).reset_index()
    agg['se'] = agg['std'] / np.sqrt(agg['count'].clip(lower=1))
    agg['lo'] = agg['mean'] - 1.96 * agg['se']
    agg['hi'] = agg['mean'] + 1.96 * agg['se']

    plt.figure(figsize=(6, 4))
    plt.plot(agg['tau'], agg['mean'], marker='o', label='Mean')
    plt.fill_between(agg['tau'], agg['lo'], agg['hi'], alpha=0.2, label='95% CI')
    plt.axvline(0, color='gray', linestyle='--', linewidth=1)
    plt.xlabel('Weeks around event')
    plt.ylabel(f'Mean {value_col}')
    plt.title(title)
    plt.legend(loc='best', frameon=False)
    plt.tight_layout()
    plt.savefig(out_path)
    plt.close()

# draw simple brand-level time-series
def plot_brand_timeseries(panel: pd.DataFrame,
                          company: str,
                          left_col: str = 'num_articles',
                          right_col: str = 'num_memes_z',
                          smooth: int = 0,
                          out_path: str | None = None):
    g = panel.loc[panel['company'] == company].sort_values('week_start').copy()
    if g.empty:
        return

    x = g['week_start']
    left = g[left_col].astype(float)
    right = g[right_col].astype(float)

    if smooth and smooth > 1:
        left = left.rolling(window=smooth, min_periods=1).mean()
        right = right.rolling(window=smooth, min_periods=1).mean()

    fig, ax1 = plt.subplots(figsize=(9, 3.5))
    color_left, color_right = '#1f77b4', '#d62728'

    ax1.plot(x, left, color=color_left, label=left_col)
    ax1.set_xlabel('Week')
    ax1.set_ylabel(left_col, color=color_left)
    ax1.tick_params(axis='y', labelcolor=color_left)

    ax2 = ax1.twinx()
    ax2.plot(x, right, color=color_right, label=right_col, alpha=0.85)
    ax2.set_ylabel(right_col, color=color_right)
    ax2.tick_params(axis='y', labelcolor=color_right)

    fig.suptitle(f'{company}: {left_col} vs {right_col}')
    fig.tight_layout()

    if out_path:
        plt.savefig(out_path)
        plt.close()
    else:
        plt.show()


In [22]:
# Descriptive brand time-series and event-study with 95% CIs

# Select a few brands (top by NYT articles)
top_companies = (
    panel.groupby('company')['NYT_mention']
            .sum()
            .sort_values(ascending=False)
            .head(6)
            .index
            .tolist()
)

for c in top_companies:
    out = os.path.join(FIG_DIR, f"ts_{_sanitize_filename(c)}_num_articles_vs_num_memes_z.png")
    plot_brand_timeseries(panel, c, left_col='NYT_mention', right_col='num_memes_z', smooth=4, out_path=out)
    print(f"Saved {out}")

# Event-study plots with 95% confidence intervals
plot_event_ci(pos_z, 'num_memes_z',
              'Event: Positive news vs normalized (z) meme volume (95% CI)',
              os.path.join(FIG_DIR, 'event_pos_num_memes_z_ci.png'))
plot_event_ci(neg_z, 'num_memes_z',
              'Event: Negative news vs normalized (z) meme volume (95% CI)',
              os.path.join(FIG_DIR, 'event_neg_num_memes_z_ci.png'))
print('Saved CI event-study plots.')


Saved /Users/beszabo/bene/szakdolgozat/figures/ts_facebook_num_articles_vs_num_memes_z.png
Saved /Users/beszabo/bene/szakdolgozat/figures/ts_youtube_num_articles_vs_num_memes_z.png
Saved /Users/beszabo/bene/szakdolgozat/figures/ts_netflix_num_articles_vs_num_memes_z.png
Saved /Users/beszabo/bene/szakdolgozat/figures/ts_google_num_articles_vs_num_memes_z.png
Saved /Users/beszabo/bene/szakdolgozat/figures/ts_instagram_num_articles_vs_num_memes_z.png
Saved /Users/beszabo/bene/szakdolgozat/figures/ts_apple_num_articles_vs_num_memes_z.png
Saved CI event-study plots.


In [23]:
# --- Z-score event study on monthly panel ---
from lead_lag_normalized import event_study_value, add_normalizations

PANEL_MONTHLY = os.path.join(PANEL_DIR, "company_monthly_panel.csv")
panel_m = pd.read_csv(PANEL_MONTHLY, parse_dates=["month_start"])
panel_m = add_normalizations(panel_m, monthly=True)

# Compute event windows where positive / negative sentiment events are top 10 % of their distribution
pos_z_m, neg_z_m = event_study_value(panel_m,
                                     pos_feature="mean_pos",
                                     neg_feature="mean_neg",
                                     value_col="num_memes_z",
                                     window=3,
                                     monthly=True)
plot_event(pos_z_m, "num_memes_z",
           "Event (monthly): Positive news vs normalized meme volume",
           os.path.join(FIG_DIR, "event_pos_num_memes_z_monthly.png"))
plot_event(neg_z_m, "num_memes_z",
           "Event (monthly): Negative news vs normalized meme volume",
           os.path.join(FIG_DIR, "event_neg_num_memes_z_monthly.png"))
print("Monthly z-score event-study plots saved.")

df_pos, df_neg = event_study(panel_m, pos_feature='mean_pos', neg_feature='mean_neg', window=3)
plot_event(df_pos, 'meme_spike', 'Event: Positive news vs meme_spike (raw)', os.path.join(FIG_DIR, 'event_pos_meme_spike_raw_monthly.png'))
plot_event(df_neg, 'meme_spike', 'Event: Negative news vs meme_spike (raw)', os.path.join(FIG_DIR, 'event_neg_meme_spike_raw_monthly.png'))

FileNotFoundError: [Errno 2] No such file or directory: '/Users/beszabo/bene/szakdolgozat/data/panels/company_monthly_panel.csv'

In [9]:
! uv add U transformers
! python - <<'PY' /
from huggingface_hub import snapshot_download
snapshot_download("yuval-pinter/visual-sentiment-clip",
                  local_dir="models/visual-sentiment",
                  repo_type="model", revision="main")
PY

Resolved 141 packages in 1.97s                                       
Prepared 1 package in 279ms                                              ⠋ Preparing packages... (0/0)                                                   
Installed 1 package in 3ms                                  
 + u==1.0.4


RepositoryNotFoundError: 401 Client Error. (Request ID: Root=1-69088a0f-170f37d2192295361d114445;b9ed3d5e-6acd-4405-b3c8-6c67879d6057)

Repository Not Found for url: https://huggingface.co/api/models/yuval-pinter/visual-sentiment-clip/revision/main.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated. For more details, see https://huggingface.co/docs/huggingface_hub/authentication
Invalid username or password.

In [ ]:
from PIL import Image
import torch, torch.nn.functional as F
from transformers import CLIPProcessor, CLIPModel

model_id = "victorandrees/visual-sentiment"
proc   = CLIPProcessor.from_pretrained(model_id)
model  = CLIPModel.from_pretrained(model_id)

def img_sentiment(path):
    img = Image.open(path).convert("RGB")
    inputs = proc(text=["negative", "positive"], images=img,
                  return_tensors="pt", padding=True)
    with torch.no_grad():
        logits = model(**inputs).logits_per_image.squeeze()   # [neg,pos]
    probs = F.softmax(logits, dim=0).tolist()
    return {"neg": probs[0], "pos": probs[1], "score": probs[1]-probs[0]}
print(img_sentiment("data/prediction_images/adidas/meme_submissions_1487127.png"))



OSError: victorandrees/visual-sentiment is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`